In [8]:
import sys
print(sys.path)

['/Users/bhalbur/anaconda3/lib/python36.zip', '/Users/bhalbur/anaconda3/lib/python3.6', '/Users/bhalbur/anaconda3/lib/python3.6/lib-dynload', '', '/Users/bhalbur/anaconda3/lib/python3.6/site-packages', '/Users/bhalbur/anaconda3/lib/python3.6/site-packages/aeosa', '/Users/bhalbur/anaconda3/lib/python3.6/site-packages/IPython/extensions', '/Users/bhalbur/.ipython']


# This notebook for testing code
#### The final version will be saved as indivudual python files that can be executed on the terminal or called by another python program

In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser

In [ ]:
# Initialize PyMongo to work with MongoDBs
client = pymongo.MongoClient('mongodb://localhost:27017')

In [ ]:
# Define database and collection
db = client.mars
collection = db.marspage

In [2]:
## PART 1 Mars News
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
response = requests.get(url, timeout=5)

In [3]:
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'html.parser')

In [27]:
header = soup.find('div', class_='content_title')
body = soup.find('div', class_='rollover_description_inner')
print(header.text)
print(body.text)
news_dict = {}
news_dict['article_title'] = header.text.strip()
news_dict['article_content'] = body.text.strip()
print(news_dict)



NASA's InSight Places First Instrument on Mars



In deploying its first instrument onto the surface of Mars, the lander completes a major mission milestone.

{'article_title': "NASA's InSight Places First Instrument on Mars", 'article_content': 'In deploying its first instrument onto the surface of Mars, the lander completes a major mission milestone.'}


In [26]:
## Part 1 News -- WITH SPLINTER
news_url = 'https://mars.nasa.gov/news/'
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=True)
browser.visit(news_url)
news_soup = BeautifulSoup(browser.html, 'html.parser')
print(len(browser.html))
header = news_soup.find('div', class_='content_title')
print(header.text)
body = news_soup.find('div', class_='rollover_description_inner')
print(body.text)
news_dict = {}
news_dict['type'] = 'news'
news_dict['title'] = header.text.strip()
news_dict['content'] = body.text.strip()
print(news_dict)
#db.variables.insert_one(news_dict)

1210751
Beyond Mars, the Mini MarCO Spacecraft Fall Silent
The first CubeSat mission to deep space may have reached their limit, but they could inspire future NASA missions.
{'type': 'news', 'title': 'Beyond Mars, the Mini MarCO Spacecraft Fall Silent', 'content': 'The first CubeSat mission to deep space may have reached their limit, but they could inspire future NASA missions.'}


In [ ]:
### PART 2 FEATURED IMAGE

In [11]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=True)

In [15]:
featured_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(featured_url)
browser.click_link_by_partial_text('FULL IMAGE')

In [16]:
featured_html = browser.html
print(len(featured_html), 246791)
featured_soup = BeautifulSoup(featured_html, 'html.parser')
url_suffix = featured_soup.find('img', class_='fancybox-image').attrs['src']
featured_img_url = 'https://www.jpl.nasa.gov' + url_suffix
print(f'Featured img url: {featured_img_url}')

254024 246791
Featured img url: https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16837_ip.jpg


In [5]:
type(featured_soup.find('img', class_='fancybox-image'))

bs4.element.Tag

In [50]:
featured_dict = {}
featured_dict['title'] = 'Featured_Image'
featured_dict['img_url'] = featured_img_url
print(featured_dict)

{'title': 'Featured_Image', 'img_url': 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars/spaceimages/images/mediumsize/PIA17845_ip.jpg'}


In [ ]:
### PART 3 TWITTER SCRAPING
twitter_url='https://twitter.com/marswxreport?lang=en'

In [ ]:
twit_resp = requests.get(twitter_url, timeout=5)

In [ ]:
twitter_soup = BeautifulSoup(twit_resp.text, 'html.parser')

In [ ]:
tweets = twitter_soup.find_all('p', class_='TweetTextSize')
for tweet in tweets:
    if tweet.text[:3] == 'Sol':
        weather_dict = {}
        weather_dict['weather'] = tweet.text
        print(tweet.text)
        break
    else: next

In [47]:
## PART 4 MARS FACTS
fact_url = 'https://space-facts.com/mars/'
fact_resp = requests.get(fact_url, timeout=5)

In [48]:
fact_soup = BeautifulSoup(fact_resp.text, 'html.parser')
facts = fact_soup.find_all('tr')
for fact in facts:
    print(fact.text)


Equatorial Diameter:6,792 km



Polar Diameter:6,752 km



Mass:6.42 x 10^23 kg (10.7% Earth)


Moons:2 (Phobos & Deimos)


Orbit Distance:227,943,824 km (1.52 AU)


Orbit Period:687 days (1.9 years)



Surface Temperature: -153 to 20 °C


First Record:2nd millennium BC


Recorded By:Egyptian astronomers



In [49]:
#this version prints all the facts into one dictionary
fact_dict = {}
for fact in facts:
    description, value = fact.text.strip().split(':')
    fact_dict[description] = value    
print(fact_dict)

{'Equatorial Diameter': '6,792 km', 'Polar Diameter': '6,752 km', 'Mass': '6.42 x 10^23 kg (10.7% Earth)', 'Moons': '2 (Phobos & Deimos)', 'Orbit Distance': '227,943,824 km (1.52 AU)', 'Orbit Period': '687 days (1.9 years)', 'Surface Temperature': ' -153 to 20 °C', 'First Record': '2nd millennium BC', 'Recorded By': 'Egyptian astronomers'}


In [ ]:
#this version puts the facts into a list of individual dictionaries
fact_list = []
for fact in facts:
    fact_dict = {}
    description, value = fact.text.strip().split(':')
    fact_dict[description] = value
    fact_list.append(fact_dict)
    ##insert to Mongo DB
print(fact_list)

In [ ]:
###Part 5 Mars Hemispheres

In [35]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [36]:
hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemi_url)

In [38]:
hemispheres = ['Cerberus','Schiaparelli','Syrtis','Valles']
hemi_list = []
for hemi in hemispheres:
    hemi_dict = {}
    browser.click_link_by_partial_text(hemi)
    hemi_soup = BeautifulSoup(browser.html, 'html.parser')
    hemi_title = hemi_soup.find('h2', class_='title')
    hemi_title = hemi_title.text.split(' Enhanced')[0]
    print(hemi_title)
    dl_tag = hemi_soup.find('div', class_='downloads')
    hemi_img = dl_tag.find('a').attrs['href']
    print(hemi_img)
    hemi_dict['title'] = hemi_title
    hemi_dict['img_url'] = hemi_img
    hemi_list.append(hemi_dict)
    browser.back()
print(hemi_list)

Cerberus Hemisphere
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
Schiaparelli Hemisphere
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
Syrtis Major Hemisphere
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
Valles Marineris Hemisphere
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
[{'title': 'Cerberus Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology

In [ ]:
### LEFTOVERS FROM HOCKEY ASSIGNMENT
headers = soup.find_all('li', class_='mixed-feed__item--article')
for h in headers:
    try:
        title = h.find('h4').text
        sub = h.find('h5').text
        logo = h.find('img', class_='logo').attrs['src']
        print(title)
        print(' - '+sub)
        print(f"https://{logo}")
        print('-----------------')
        
        article = {
            'title': title,
            'subtitle': sub,
            'team_logo': logo 
        }
              
        collection.insert_one(article)
              
    except Exception as e: 
        print(e)

In [ ]:
# Display the MongoDB records created above
articles = db.articles.find()
for article in articles:
    print(article)

In [28]:
import pandas as pd

In [30]:
df = pd.DataFrame([{'Equatorial Diameter': '6,792 km'}, {'Polar Diameter': '6,752 km'}, {'Mass': '6.42 x 10^23 kg (10.7% Earth)'}, {'Moons': '2 (Phobos & Deimos)'}, {'Orbit Distance': '227,943,824 km (1.52 AU)'}, {'Orbit Period': '687 days (1.9 years)'}, {'Surface Temperature': ' -153 to 20 °C'}, {'First Record': '2nd millennium BC'}, {'Recorded By': 'Egyptian astronomers'}])

In [31]:
df.head()

,Equatorial Diameter,First Record,Mass,Moons,Orbit Distance,Orbit Period,Polar Diameter,Recorded By,Surface Temperature
0,"6,792 km",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,"6,752 km",NaN,NaN
2,NaN,NaN,6.42 x 10^23 kg (10.7% Earth),NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,2 (Phobos & Deimos),NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,"227,943,824 km (1.52 AU)",NaN,NaN,NaN,NaN


In [39]:
fact_url = 'https://space-facts.com/mars/'
fact_resp = requests.get(fact_url, timeout=15)
fact_soup = BeautifulSoup(fact_resp.text, 'html.parser')
facts = fact_soup.find_all('tr')
fact_list = []
value_list = []
fact_dict = {}
fact_dict['type'] = 'facts'
for fact in facts:
    description, value = fact.text.strip().split(':')
    fact_list.append(description)
    value_list.append(value)
    content_dict = {}
    content_dict
    fact_dict['content'] = fact_list
    fact_dict['values'] = value_list
print(fact_dict['content'], fact_dict['values'])

['Equatorial Diameter', 'Polar Diameter', 'Mass', 'Moons', 'Orbit Distance', 'Orbit Period', 'Surface Temperature', 'First Record', 'Recorded By'] ['6,792 km', '6,752 km', '6.42 x 10^23 kg (10.7% Earth)', '2 (Phobos & Deimos)', '227,943,824 km (1.52 AU)', '687 days (1.9 years)', ' -153 to 20 °C', '2nd millennium BC', 'Egyptian astronomers']


In [51]:
df = pd.DataFrame([fact_dict])

In [52]:
df.head()

,Equatorial Diameter,First Record,Mass,Moons,Orbit Distance,Orbit Period,Polar Diameter,Recorded By,Surface Temperature
0,"6,792 km",2nd millennium BC,6.42 x 10^23 kg (10.7% Earth),2 (Phobos & Deimos),"227,943,824 km (1.52 AU)",687 days (1.9 years),"6,752 km",Egyptian astronomers,-153 to 20 °C
